In [6]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [7]:
import numpy as np
import random
from multiprocessing import Pool

# Lectura de datos
data = np.loadtxt('/content/drive/MyDrive/MÁSTER/IC/P2/data/tai256c.dat', skiprows=1)
flow = np.int32(data[:256])
distances = np.int32(data[256:])

seed = 2024
random.seed(seed)
np.random.seed(seed)

In [8]:
# Generación de la población inicial
def generate_population(population_size, n=256):
    return np.array([np.random.permutation(np.arange(n)) for _ in range(population_size)])

def fitness_pop(population, flow, distances):
    return np.sum(flow[np.newaxis, :, :] * distances[population[:, :, np.newaxis], population[:, np.newaxis, :]],
                  axis=(1, 2))

In [9]:
# Función de selección por torneo
def tournament_selection(population, fitness_vals, tournament_size=5):
    selected = []
    for _ in range(len(population)):
        tournament = random.sample(range(len(population)), tournament_size)
        winner = tournament[np.argmin([fitness_vals[i] for i in tournament])]
        selected.append(population[winner])
    return np.array(selected)

In [10]:
# Operador de cruce por orden (OX)
def ox_crossover(parent1, parent2):
    size = len(parent1)
    start, end = sorted(np.random.choice(range(size), 2, replace=False))
    child = -np.ones(size, dtype=int)
    child[start:end+1] = parent1[start:end+1]
    idx = 0
    for i in range(size):
        if child[i] == -1:
            while parent2[idx] in child:
                idx += 1
            child[i] = parent2[idx]
    return child

In [11]:
# Mutación por intercambio (Swap)
def swap_mutation(individual, mutation_prob=0.05):
    if random.random() < mutation_prob:
        i, j = np.random.choice(range(len(individual)), 2, replace=False)
        individual[i], individual[j] = individual[j], individual[i]
    return individual

In [12]:
def hill_climbing_optimized(individual, flow, distances, max_iterations=100):
    current = individual.copy()
    current_fitness = fitness_pop(np.array([current]), flow, distances)[0]
    for _ in range(max_iterations):
        i, j = np.random.choice(range(len(current)), 2, replace=False)
        neighbor = current.copy()
        neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
        # Cálculo delta:
        delta_fitness = calculate_fitness_delta(current, i, j, flow, distances)
        if current_fitness + delta_fitness < current_fitness:
            current = neighbor
            current_fitness += delta_fitness
    return current, current_fitness

def calculate_fitness_delta(individual, i, j, flow, distances):
    n = len(individual)
    delta = 0
    for k in range(n):
        if k != i and k != j:
            delta += (flow[i, k] - flow[j, k]) * (distances[individual[j], individual[k]] - distances[individual[i], individual[k]])
            delta += (flow[k, i] - flow[k, j]) * (distances[individual[k], individual[j]] - distances[individual[k], individual[i]])
    return delta

In [13]:
# Algoritmo genético con variante baldwiniana
def baldwinian_genetic_algorithm(flow, distances, population_size=1000, generations=500, crossover_prob=0.9, mutation_prob=0.05, tournament_size=5):
    population = generate_population(population_size)
    pool = Pool()  # Crear un pool de procesos para paralelizar

    best_fitness = float('inf')
    generations_without_improvement = 0
    max_generations_without_improvement = 25

    for generation in range(generations):
        # Aplicar hill climbing a los mejores individuos
        top_k = int(0.1 * population_size)
        fitness_vals = fitness_pop(population, flow, distances)
        top_individuals = population[np.argsort(fitness_vals)[:top_k]]
        results = pool.starmap(hill_climbing_optimized, [(ind, flow, distances) for ind in top_individuals])
        hill_climbed_population = np.array([res[0] for res in results])
        hill_climbed_fitness = np.array([res[1] for res in results])

        # Combinar con el resto de la población
        remaining_population = population[top_k:]
        fitness_vals[top_k:] = fitness_pop(remaining_population, flow, distances)

        # Selección
        population = tournament_selection(np.vstack((hill_climbed_population, remaining_population)), fitness_vals, tournament_size)

        # Cruzamiento
        offspring = []
        for i in range(0, population_size, 2):
            if random.random() < crossover_prob:
                child1 = ox_crossover(population[i], population[i+1])
                child2 = ox_crossover(population[i+1], population[i])
            else:
                child1, child2 = population[i], population[i+1]
            offspring.append(child1)
            offspring.append(child2)
        population = np.array([swap_mutation(ind, mutation_prob) for ind in offspring])

        # Imprimir mejor costo
        best_idx = np.argmin(fitness_vals)
        print(f"Generación {generation+1}: Mejor coste = {fitness_vals[best_idx]}")

        #Finalizar si no hay mejora
        if generations_without_improvement >= max_generations_without_improvement:
           print(f"Terminando en la generación {generation+1} debido a la falta de mejora.")
           break

    best_idx = np.argmin(fitness_vals)
    return population[best_idx], fitness_vals[best_idx]

In [14]:
# Ejecución del algoritmo
best_solution_baldwin, best_cost_baldwin = baldwinian_genetic_algorithm(flow, distances)

print("\nMejor solución encontrada (Baldwiniano):", best_solution_baldwin)
print("Coste asociado a la mejor solución (Baldwiniano):", best_cost_baldwin)

Generación 1: Mejor coste = 49640646
Generación 2: Mejor coste = 49213330
Generación 3: Mejor coste = 48351654
Generación 4: Mejor coste = 48133662
Generación 5: Mejor coste = 47973696
Generación 6: Mejor coste = 47797634
Generación 7: Mejor coste = 47798554
Generación 8: Mejor coste = 47570356
Generación 9: Mejor coste = 47083684
Generación 10: Mejor coste = 47327920
Generación 11: Mejor coste = 47356492
Generación 12: Mejor coste = 47286926
Generación 13: Mejor coste = 46960938
Generación 14: Mejor coste = 46960938
Generación 15: Mejor coste = 47028606
Generación 16: Mejor coste = 46816684
Generación 17: Mejor coste = 46737462
Generación 18: Mejor coste = 46668744
Generación 19: Mejor coste = 46635982
Generación 20: Mejor coste = 46498414
Generación 21: Mejor coste = 46301458
Generación 22: Mejor coste = 46301458
Generación 23: Mejor coste = 46294526
Generación 24: Mejor coste = 46238342
Generación 25: Mejor coste = 46187076
Generación 26: Mejor coste = 46098244
Generación 27: Mejor 